# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2:1b"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

In [5]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has a wide range of business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can be used to create high-quality content such as articles, social media posts, and product descriptions. Companies like Medium and LinkedIn use generative models to create engaging content for their users.
2. **Marketing Automation**: Generative AI can help businesses personalize customer interactions by generating targeted messages and offers based on user behavior and preferences.
3. **Predictive Maintenance**: Generative AI can analyze data from sensors and machines to predict maintenance needs, reducing downtime and increasing efficiency in industries like manufacturing and logistics.
4. **Design and Product Development**: Generative AI can be used to generate new product designs, prototypes, and even entire products based on specific requirements and constraints.
5. **Sales and Customer Service**: Generative AI-powered chatbots and vi

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [6]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, enabling companies to create innovative products, services, and experiences. Here are some examples:

1. **Product Design**: Generative AI can design products with unprecedented levels of complexity, such as 3D models, product prototypes, and even entire manufacturing systems. Companies like IKEA, Mercedes-Benz, and Boeing are using generative AI to optimize their designs and reduce production costs.
2. **Content Creation**: Generative AI can generate high-quality content, including articles, videos, and social media posts. This reduces the workload for human creators, allowing companies like BuzzFeed, The New York Times, and LinkedIn to produce vast amounts of content at scale.
3. **Marketing Automation**: Generative AI can help businesses personalize customer interactions, predict customer behavior, and optimize marketing campaigns. Companies like Amazon, Facebook, and Target are using generative AI to automa

## Alternative approach - using OpenAI python library to connect to Ollama

In [7]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has a wide range of business applications across various industries, including:

1. **Customer Service**: Chatbots and virtual assistants powered by generative models can help automate customer support by generating responses to common queries, freeing up human customer support agents to focus on more complex issues.

2. **Content Creation**: Generative AI can be used to generate high-quality content such as articles, social media posts, and product descriptions without the need for extensive human intervention. This can help businesses save time and resources while increasing productivity.

3. **Marketing and Advertising**: Generative models can create personalized marketing messages, product recommendations, and target audience personas, enabling businesses to optimize their marketing strategies and improve conversion rates.

4. **Design and Product Development**: Generative AI can generate new designs, patterns, and shapes by simulating the creative process of human de

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [8]:
!ollama pull deepseek-r1:1.5b

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest 
pulling aabd4debf0c8:   0% â–•                  â–� 3.0 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% â–•                  â–�  10 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% â–•                  â–�  16 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% â–•                  â–�  24 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   3% â–•                  â–�  32 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   3% â–•                  â–�  36 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   4% â–•                  â–�  44 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   5% â–•                  â–�  51 MB/1.1 GB

In [9]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

<think>
Okay, I need to figure out how to answer the user's query about LLMs. They asked for definitions of some core concepts related to them, specifically focusing on neural networks, attention, and transformers.

First, I'll start by remembering my own knowledge about these topics. LLMs are large language models that can understand and generate human-level text. They work by processing vast amounts of text data through a neural network.

I know a neural network is made up of layers composed of neurons. Each neuron processes information from its inputs, applies an activation function, and passes the result onto the next layer. This process is inspired by the human brain's biological functions. In LLMs, these networks are specialized for processing text sequences because they have mechanisms like attention and transformers.

Attention is about highlighting key points in input data without focusing on specific words. It probably refers to how models identify important tokens or positio

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [10]:
# Step 1: Create your prompts

system_prompt = "you are a personal assistant that write email replies in a professional manner but ultimately the answer should be no thank you."
user_prompt = """
Please draft a response to the following email: Hey Ben,

We recently launched a new platform at company A that can transform how your team work, and we’re seeing awesome  results for your target industry.

Do you handle data protection? If not, who is that person?

Thanks,

Jon Doe
"""

# Step 2: Make the messages list

messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]# fill this in

# Step 3: Call OpenAI

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

# Step 4: print the result

print(response)

Here's a draft response:

Dear Jon,

Thank you for reaching out to us about our new platform. We're thrilled to hear about the positive impact it's having in the target industry.

Regarding your question, I'm afraid we do not have an internal data protection specialist who can review and ensure compliance with relevant regulations. Our team is responsible for developing and implementing security measures, but we rely on external experts to provide guidance on data protection best practices.

I've forwarded your inquiry to our IT department, as they will be able to provide you with more specific information about our platform's data handling policies. They can also help address any concerns you may have regarding data protection compliance.

If you have any further questions or would like to discuss this in more detail, please don't hesitate to reach out.

Best regards,
Ben
model='llama3.2:1b' created_at='2025-07-02T12:42:21.3846187Z' done=True done_reason='stop' total_duration=11017610

In [11]:
# Step 1: Create your prompts

system_prompt = "You are a genius assistant that explains things in a direct way"
user_prompt = """
What does the following python code mean and do? yield from set(book.get("author") for book in books if book.get("author"))

"""

# Step 2: Make the messages list

messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]# fill this in

# Step 3: Call OpenAI

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

# Step 4: print the result

print(response)

Let's break down this Python code step by step.

**Code Explanation:**

The given code is using a technique called "generator expression" or "iterable expression". It's designed to iterate over a collection of objects (in this case, `books`) and yield each object one at a time.

Here's what the code does:

```python
yield from set(book.get("author") for book in books if book.get("author"))
```

* `book.get("author")` is a function that retrieves the "author" attribute from each `book` object. If the attribute doesn't exist, it returns `None`.
* The expression inside the parentheses `()`: this is another generator expression.
	+ It's iterating over each `book` in the `books` collection.
	+ For each book, it calls `book.get("author")`, which might return `None`. If not, it yields the author name (which could be a string).
* The `yield from` syntax is used to "pass" the generator expression's results back to the outer code. Think of it as "returning the yielded values".
* In this case, th